## 神经根插值算法

### 导入需要的包

In [1]:
import os
import numpy as np
import vtk
import pyvista as pv
from pyvistaqt import BackgroundPlotter
import pandas as pd
import math

### 导入模型文件和标注点

In [2]:
pv.set_jupyter_backend('trame')  

In [4]:
from pyvistaqt import BackgroundPlotter



def readstl_pv(filepath):

    meshes = pv.read(stl_files)

    plotter = BackgroundPlotter()

    for mesh in meshes:

        plotter.add_mesh(mesh)

    plotter.show()



stl_files = [r'YOUR_DATA_PATH/Spinal Cord.stl', r'YOUR_DATA_PATH/T11.stl']


readstl_pv(stl_files)

In [42]:
import pandas as pd
IF_L_Tbl = pd.DataFrame.from_dict(pd.read_json(r'YOUR_DATA_PATH\IF_L.mrk.json')['markups'][0]['controlPoints'])
IF_R_Tbl = pd.DataFrame.from_dict(pd.read_json(r'YOUR_DATA_PATH\IF_R.mrk.json')['markups'][0]['controlPoints'])
DREP_L_Tbl = pd.DataFrame.from_dict(pd.read_json(r'YOUR_DATA_PATH\DREP_L.mrk.json')['markups'][0]['controlPoints'])
DREP_R_Tbl = pd.DataFrame.from_dict(pd.read_json(r'YOUR_DATA_PATH\DREP_R.mrk.json')['markups'][0]['controlPoints'])

In [43]:

# if_L
if_L = np.array([np.array(i) for i in IF_L_Tbl['position']])
DREP_L = np.array([np.array(i) for i in DREP_L_Tbl['position']])


# poly_data = pv.PolyData(if_L)

# Visualize the fiducial points
plotter = pv.Plotter()
#plotter = BackgroundPlotter()
# plotter.add_mesh(poly_data, color='red', point_size=10, render_points_as_spheres=True) # no need if use add_point_labels


plotter.add_point_labels(if_L, list(IF_L_Tbl['label']), font_size=24, point_color='red', point_size=20, render_points_as_spheres=True, always_visible=True)
plotter.add_point_labels(DREP_L, list(DREP_L_Tbl['label']), font_size=24, point_color='green', point_size=20, render_points_as_spheres=True, always_visible=True)

#import spinal cord and vertebrae
stl_files = [r'YOUR_DATA_PATH/Segmentation-models_1/Spinal Cord.stl', r'YOUR_DATA_PATH/Segmentation-models_1/T11.stl']

meshes = pv.read(stl_files)

#plotter = BackgroundPlotter()

for mesh in meshes:

        plotter.add_mesh(mesh)

plotter.show()

Widget(value="<iframe src='http://localhost:59450/index.html?ui=P_0x1f557739210_2&reconnect=auto' style='width…

### 切面识别中心和轮廓

In [17]:
# 在DREP高度切片
sliced_mesh = meshes[0].slice(normal='z', origin=DREP_L_Tbl['position'][0])
center = np.array(sliced_mesh.center)

In [59]:

#import spinal cord and vertebrae

# Create a plotter
#plotter = pv.Plotter()
plotter = BackgroundPlotter()
plotter.add_mesh(meshes[0])
# 在DREP高度切片
sliced_mesh = meshes[0].slice(normal='z', origin=DREP_L_Tbl['position'][0])
# Visualization
plotter.add_mesh(sliced_mesh, color = 'red')
plotter.add_mesh(np.array(interp_list), color='blue')
#plotter.add_mesh(pv.Line(center,cart2polTransform(np.array([20,0,0]),center,direction= 'pol2cart')))
#plotter.add_mesh(pv.Line(center,cart2polTransform(np.array([20,math.pi/2,0]),center,direction= 'pol2cart')))
plotter.add_point_labels(if_L[2], [IF_L_Tbl['label'][2]], font_size=24, point_color='red', point_size=20, render_points_as_spheres=True, always_visible=True)
plotter.add_point_labels(DREP_L, list(DREP_L_Tbl['label']), font_size=24, point_color='green', point_size=20, render_points_as_spheres=True, always_visible=True)


plotter.show()

### 坐标系转换

In [10]:
import math
def cart2polTransform(coordinate, origin = np.array([0,0,0]), direction='cart2pol'):
    ### Parameters:
    # coordinate: 3D array needs to transform
    # origin(deprecated): the origin of polar coordinate
    # direction: the direction of transform 'cart2pol','pol2cart'
    # 

    if direction == 'cart2pol':
        x = coordinate[0]-origin[0]
        y = coordinate[1]-origin[1]
        z = coordinate[2]-origin[2]
        pol = []
        pol.append((x**2+y**2)**0.5)
        a = math.atan2(y,x)
        pol.append(a if a>=0 else 2*math.pi+a)
        pol.append(z)
        return np.array(pol)
            
        
    elif direction == 'pol2cart':
        x = coordinate[0]
        y = coordinate[1]
        z = coordinate[2]
        cart = []
        cart.append(x*math.cos(y))
        cart.append(x*math.sin(y))
        cart.append(z)
        
        return np.array(cart)+origin
        
        
    else:
        raise Exception(" Wrong direction parameter input, should be 'cart2pol' or 'pol2cart'.")


### 获得脊髓半径

In [11]:
def get_dwm(angle_rad, center):
    # Parameters:
    # angle : the radian angle of the interpolate points

    # Define the direction vector of the ray
    # angle_rad = angle-
    direction = np.array([np.cos(angle_rad), np.sin(angle_rad), 0])

    # Define the length of the ray
    ray_length = 20

    # Calculate the end point of the ray
    end_raypoint = np.array(center) + ray_length * direction

    # Find the closest point on the slice to the rayend
    closest_point = sliced_mesh.find_closest_point(end_raypoint)
    
    # Calculate the distance
    d = np.sqrt(np.sum((center-sliced_mesh.points[closest_point])**2))
    
    return d

### 插值函数

In [12]:

def interpolate_point_sampling(num,origin,entrypoint,endpoint,gapSC):
    # Parameters：
    # num： the number of sampling points to interpolate(including the enterpoint and endpoint)
    # entrypoint: the 3-D array of enrtypoint()
    #
    # Varaibles:
    # z: the height of the point(polar/cartesian coordinate)
    # a: the angle of the point(polar coordinate relative to the DREP height slice center)
    # d: the radius of the point(polar coordinate relative to the DREP height slice center)
    
    # Coordinate system transform
    etrp_pol = cart2polTransform(entrypoint,origin)
    endp_pol = cart2polTransform(endpoint,origin)
    d,a,z =[],[],[]
    
    #初始化角度
    ai = etrp_pol[1]
    for i in range(2,num):
        # 第一段规则
        if ai <= math.pi:
            zi = etrp_pol[2] + (i/(num-1))*(endp_pol[2] - etrp_pol[2])
            ai = etrp_pol[1] + (i/(num-1))*(endp_pol[1] - etrp_pol[1])
            dwmi = get_dwm(ai,origin)
            di = dwmi + gapSC
            i0 = i
            di0 = di
            ai0 = ai
            zi0 = zi
            
        #第二段规则，直接向椎间孔
        else:
            # wi比例因子
            wi = (i-i0)/(num-i0)
            di = (1-wi)*di0+wi*endp_pol[0]
            ai = (1-wi)*ai0+wi*endp_pol[1]
            zi = (1-wi)*zi0+wi*endp_pol[2]
        d.append(di)
        a.append(ai)
        z.append(zi)
        
        
        
    return d,a,z

In [18]:
d,a,z = interpolate_point_sampling(30,center,DREP_L[0],if_L[2],0.1)

In [19]:
for i in zip(d,a,z):
    print(i)

(4.538941056699068, 1.9304471154770748, -5.557416996285752)
(4.679626097993681, 1.996026133054288, -8.336125494428627)
(4.679626097993681, 2.0616051506315016, -11.114833992571503)
(4.8446550576742515, 2.127184168208715, -13.89354249071438)
(4.8579131430326195, 2.192763185785928, -16.672250988857254)
(4.8579131430326195, 2.2583422033631417, -19.45095948700013)
(4.867775551460844, 2.3239212209403552, -22.229667985143006)
(4.917525496706888, 2.3895002385175688, -25.008376483285883)
(4.917525496706888, 2.4550792560947823, -27.78708498142876)
(4.917525496706888, 2.5206582736719954, -30.56579347957163)
(4.917525496706888, 2.586237291249209, -33.34450197771451)
(4.917525496706888, 2.6518163088264224, -36.123210475857384)
(4.917525496706888, 2.717395326403636, -38.90191897400026)
(4.917525496706888, 2.7829743439808494, -41.68062747214314)
(4.848709817864149, 2.8485533615580625, -44.45933597028601)
(4.848709817864149, 2.9141323791352756, -47.23804446842888)
(4.737531924641851, 2.979711396712489

In [26]:
DREP_L[0]

array([-2.24930811, 52.77119827, -8.77660179])

In [20]:
interp_list=[DREP_L[0]]
for di,ai,zi in zip(d,a,z):
    interp_list.append(cart2polTransform([di,ai,zi],origin = center, direction = 'pol2cart'))
interp_list.append(if_L[2])

### 插值曲线

In [21]:
interp_list

[array([-2.24930811, 52.77119827, -8.77660179]),
 array([ -3.05346257,  53.60842671, -14.33401879]),
 array([ -3.38648091,  53.62276518, -17.11272729]),
 array([ -3.66168622,  53.48709328, -19.89143578]),
 array([ -4.01456459,  53.47381562, -22.67014428]),
 array([ -4.28638348,  53.30807959, -25.44885278]),
 array([ -4.53903244,  53.11411168, -28.22756128]),
 array([ -4.78517233,  52.91119929, -31.00626978]),
 array([ -5.0470712 ,  52.71938543, -33.78497827]),
 array([ -5.25950672,  52.47683354, -36.56368677]),
 array([ -5.45559068,  52.22088168, -39.34239527]),
 array([ -5.63448011,  51.95263023, -42.12110377]),
 array([ -5.79540596,  51.67323241, -44.89981227]),
 array([ -5.93767638,  51.38388936, -47.67852077]),
 array([ -6.06067975,  51.085845  , -50.45722926]),
 array([ -6.09800519,  50.76050231, -53.23593776]),
 array([ -6.17981211,  50.45329127, -56.01464626]),
 array([ -6.1315867 ,  50.1234605 , -58.79335476]),
 array([ -6.21653236,  49.81976109, -61.57206326]),
 array([ -6.236

In [29]:
polyline = polyline_from_points(interp_list)
polyline["scalars"] = np.arange(polyline.n_points)
tube = polyline.tube(radius=0.1)

In [20]:
def polyline_from_points(points):
    poly = pv.PolyData()
    poly.points = points
    the_cell = np.arange(0, len(points), dtype=np.int_)
    the_cell = np.insert(the_cell, 0, len(points))
    poly.lines = the_cell
    return poly


polyline = polyline_from_points(interp_list)
polyline["scalars"] = np.arange(polyline.n_points)
tube = polyline.tube(radius=0.1)
tube.plot(smooth_shading=True)

Widget(value="<iframe src='http://localhost:50138/index.html?ui=P_0x229892c3fa0_2&reconnect=auto' style='width…

In [21]:
tube

PolyData (0x229b3e5b100)
  N Cells:    1116
  N Points:   600
  N Strips:   0
  X Bounds:   -1.519e+01, -2.954e+00
  Y Bounds:   4.180e+01, 5.372e+01
  Z Bounds:   -8.694e+01, -1.432e+01
  N Arrays:   2

In [52]:

#import spinal cord and vertebrae

# Create a plotter
#plotter = pv.Plotter()
plotter = BackgroundPlotter()
plotter.add_mesh(meshes[0])
# 在DREP高度切片
sliced_mesh = meshes[0].slice(normal='z', origin=DREP_L_Tbl['position'][0])
# Visualization
#plotter.add_mesh(sliced_mesh, color = 'red')

plotter.add_mesh(np.array(interp_list), color='blue')
#plotter.add_mesh(pv.Line(center,cart2polTransform(np.array([20,0,0]),center,direction= 'pol2cart')))
#plotter.add_mesh(pv.Line(center,cart2polTransform(np.array([20,math.pi/2,0]),center,direction= 'pol2cart')))
#plotter.add_mesh(tube, color = 'yellow')
plotter.add_point_labels(if_L[2], [IF_L_Tbl['label'][2]], font_size=24, point_color='red', point_size=20, render_points_as_spheres=True, always_visible=True)
plotter.add_point_labels(DREP_L, list(DREP_L_Tbl['label']), font_size=24, point_color='green', point_size=20, render_points_as_spheres=True, always_visible=True)


plotter.show()

In [41]:
DREP_L[0][3] = 

array([ -2.24930811,  52.77119827, -14.77660179])

In [44]:
DREP_L1 = [DREP_L[0]]
for i in range(1,6):
    lastp = DREP_L1[-1].copy()
    lastp[2] = lastp[2]-1
    DREP_L1.append(lastp)

In [45]:
DREP_L1

[array([-2.24930811, 52.77119827, -8.77660179]),
 array([-2.24930811, 52.77119827, -9.77660179]),
 array([ -2.24930811,  52.77119827, -10.77660179]),
 array([ -2.24930811,  52.77119827, -11.77660179]),
 array([ -2.24930811,  52.77119827, -12.77660179]),
 array([ -2.24930811,  52.77119827, -13.77660179])]

In [ ]:
d,a,z = interpolate_point_sampling(30,center,DREP_L[0],if_L[2],0.1)

In [47]:
L1_plist = []
for i in DREP_L1:
    d,a,z = interpolate_point_sampling(30,center,i,if_L[2],0.1)
    
    interp_list=[i]
    for di,ai,zi in zip(d,a,z):
        interp_list.append(cart2polTransform([di,ai,zi],origin = center, direction = 'pol2cart'))
    interp_list.append(if_L[2])
    L1_plist.append(interp_list)

In [48]:
L1_plist

[[array([-2.24930811, 52.77119827, -8.77660179]),
  array([ -3.05346257,  53.60842671, -14.33401879]),
  array([ -3.38648091,  53.62276518, -17.11272729]),
  array([ -3.66168622,  53.48709328, -19.89143578]),
  array([ -4.01456459,  53.47381562, -22.67014428]),
  array([ -4.28638348,  53.30807959, -25.44885278]),
  array([ -4.53903244,  53.11411168, -28.22756128]),
  array([ -4.78517233,  52.91119929, -31.00626978]),
  array([ -5.0470712 ,  52.71938543, -33.78497827]),
  array([ -5.25950672,  52.47683354, -36.56368677]),
  array([ -5.45559068,  52.22088168, -39.34239527]),
  array([ -5.63448011,  51.95263023, -42.12110377]),
  array([ -5.79540596,  51.67323241, -44.89981227]),
  array([ -5.93767638,  51.38388936, -47.67852077]),
  array([ -6.06067975,  51.085845  , -50.45722926]),
  array([ -6.09800519,  50.76050231, -53.23593776]),
  array([ -6.17981211,  50.45329127, -56.01464626]),
  array([ -6.1315867 ,  50.1234605 , -58.79335476]),
  array([ -6.21653236,  49.81976109, -61.57206326

In [65]:

#import spinal cord and vertebrae

# Create a plotter
#plotter = pv.Plotter()
plotter = BackgroundPlotter()
plotter.add_mesh(meshes[0])
plotter.add_mesh(meshes[1],opacity = 0.3)
# 在DREP高度切片
sliced_mesh = meshes[0].slice(normal='z', origin=DREP_L_Tbl['position'][0])
# Visualization
#plotter.add_mesh(sliced_mesh, color = 'red')

#plotter.add_mesh(np.array(interp_list), color='')
nerveroots = pv.MultiBlock()
for i in range(0,5):
    spline = pv.Spline(L1_plist[i], 1000)
    tube = spline.tube(radius=0.1)
    # tube = spline.tube(radius=radius) # Uncomment here to change radius of the tube.
    nerveroots.append(tube)

plotter.add_mesh(nerveroots,smooth_shading=True, color = 'bisque')
#plotter.add_mesh(pv.Line(center,cart2polTransform(np.array([20,0,0]),center,direction= 'pol2cart')))
#plotter.add_mesh(pv.Line(center,cart2polTransform(np.array([20,math.pi/2,0]),center,direction= 'pol2cart')))
#plotter.add_mesh(tube, color = 'yellow')
plotter.add_point_labels(if_L[2], [IF_L_Tbl['label'][2]], font_size=24, point_color='red', point_size=20, render_points_as_spheres=True, always_visible=True)
#plotter.add_point_labels(DREP_L, list(DREP_L_Tbl['label']), font_size=24, point_color='green', point_size=20, render_points_as_spheres=True, always_visible=True)


plotter.show()